# The evolution of puffin's population regarding climate change



## 0.1 : Setting up the work environment 

- data access through the cloud MinIO Client (files are in the folder 'diffusion')
- required packages 
- organizated environment 

In [ ]:
import cdsapi
import pandas as pd
import s3fs
import zipfile
import xarray as xr
import dotenv
import os


# Creation of a folder to centralize data 
data_dir = "./data"  
os.makedirs(data_dir, exist_ok=True) 

# Creation of a folder into ./data for extracted files
temp_dir = os.path.join(data_dir, "extracted_files") # Sous-dossier d'extraction des fichiers zip
os.makedirs(temp_dir, exist_ok=True)  # Crée le dossier d'extraction


# Access information to the cloud MinIO Client (Eve's bucket)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))